In [21]:
import pygal
import cairosvg
import csv
import codecs
import ast
import csv
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import string
from pygal.style import BlueStyle

In [2]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [3]:
def lemmatize_entity(entity):
    result = (entity.lower()).translate(str.maketrans("","", string.punctuation))
    word_list = nltk.word_tokenize(result)
    filtered_output = [w for w in word_list if not w in stop_words]
    lemmatized_key = [lemmatizer.lemmatize(w) for w in filtered_output]
    return lemmatized_key

In [4]:
def read_csv(filename):
    data = []
    with (codecs.open(filename,'r',encoding="utf-8")) as f:
        filereader = csv.reader(f, delimiter=',')
        for row in filereader:
            if(''.join(row).strip()):
                data.append(row)
    f.close()
    data = data[1:]
    return data

In [5]:
data = read_csv('C:\\Users\\Reen\\Desktop\\Scripts\\Bubble\\data_my.csv')

In [6]:
len(data)

124

In [7]:
def get_graph_input(all_common, cent_crnt_dict, np_crnt_src1, np_crnt_src2,cent_src1,cent_src2,src_name):
    for key in cent_crnt_dict.keys():
        if((match_entity(all_common,key)) is False):
            #print(False,key)
            match_src1 = match_entity(np_crnt_src1,key)
            match_src2 = match_entity(np_crnt_src2,key)
            
            if(match_src1 and match_src2):
                if(src_name=="dd"):
                    all_common[key] = [cent_crnt_dict[key],0,0]
                elif(src_name=="ndtv"):
                    all_common[key] = [0,cent_crnt_dict[key],0]
                elif(src_name=="times"):
                    all_common[key] = [0,0,cent_crnt_dict[key]]
                    
            elif(match_src1 and not match_src2 ):               
                result = match_with_src(cent_src2,key) 
                if((result[0] is not None) and result[1]!=-1):
                    if(src_name=="dd"):
                        all_common[key] = [cent_crnt_dict[key],0,result[1]]
                    if(src_name=="ndtv"):
                        all_common[key] = [0,cent_crnt_dict[key],result[1]]
                    if(src_name=="times"):
                        all_common[key] = [0,result[1],cent_crnt_dict[key]]
                    
                #print("***",result,key)
                #print(match_src1,match_src2,key)
            
            elif(match_src2 and not match_src1):
                result = match_with_src(cent_src2,key) 
                if((result[0] is not None) and result[1]!=-1):
                    if(src_name=="dd"):
                        all_common[key] = [cent_crnt_dict[key],result[1],0]
                    if(src_name=="ndtv"):
                        all_common[key] = [result[1],cent_crnt_dict[key],0]
                    if(src_name=="dd"):
                        all_common[key] = [result[1],0,cent_crnt_dict[key]]

    return all_common

In [8]:
def match_with_src(src_dict,entity):
    entity_lemma = lemmatize_entity(entity)
    max_cent = -1
    max_ent = None
    for key in src_dict.keys():
        key_lemma = lemmatize_entity(key)
        if(len(key_lemma)==1 or len(entity_lemma)==1):
            if(len(list(set(key_lemma).intersection(set(entity_lemma))))>0):
                if(src_dict[key]>max_cent):
                    max_cent = src_dict[key]
                    max_ent = key
            #return [key,src_dict[key]]
        else:
            if(len(list(set(key_lemma).intersection(set(entity_lemma))))>1):
                  if(src_dict[key]>max_cent):
                    max_cent = src_dict[key]
                    max_ent = key
            #return [key,src_dict[key]]
    
    return [max_ent,max_cent]

In [9]:
len(data)

124

In [10]:
def match_entity(match_dict,entity):
    entity_lemma = lemmatize_entity(entity)
    for key in match_dict.keys():
        key_lemma = lemmatize_entity(key)
        if(len(list(set(key_lemma).intersection(set(entity_lemma))))>0):
            return True
    return False

In [11]:
#get_graph_input(all_common,dd_cent,dd_tn,dd_ndtv,ndtv_cent,times_cent,"dd")

In [23]:
for item in range(0,2):
    item_id = data[item][0]
    common_cent = ast.literal_eval(data[item][7])
    dd_cent = {x[0]: x[1] for x in ast.literal_eval(data[item][4])}
    ndtv_cent = {x[0]: x[1] for x in ast.literal_eval(data[item][5])}
    times_cent = {x[0]: x[1] for x in ast.literal_eval(data[item][6])}
    dd_tn = {x[0]: x[1] for x in ast.literal_eval(data[item][10])}
    dd_ndtv = {x[0]: x[1] for x in ast.literal_eval(data[item][11])}
    ndtv_dd = {x[0]: x[1] for x in ast.literal_eval(data[item][14])}
    ndtv_tn = {x[0]: x[1] for x in ast.literal_eval(data[item][15])}
    tn_dd = {x[0]: x[1] for x in ast.literal_eval(data[item][12])}
    tn_ndtv = {x[0]: x[1] for x in ast.literal_eval(data[item][13])}
    all_common = {key: common_cent[key] for key in common_cent.keys()}
    all_common = get_graph_input(all_common,dd_cent,dd_tn,dd_ndtv,ndtv_cent,times_cent,"dd")
    all_common = get_graph_input(all_common,ndtv_cent,ndtv_dd,ndtv_tn,dd_cent,times_cent,"ndtv")
    all_common = get_graph_input(all_common,times_cent,tn_dd,tn_ndtv,dd_cent,ndtv_cent,"times")
    #print(all_common)
    print(item)
    
    """labels = []
    dd = []
    ndtv = []
    times = []
    
    for key in all_common.keys():
        labels.append(key)
        dd.append(all_common[key][0])
        ndtv.append(all_common[key][1])
        times.append(all_common[key][2])"""
    
    labels = ['DD','NDTV','Times']
    
        
    
    cht_nm = "" + str(item_id) + ".png"
    
    """dot_chart = pygal.Dot(x_label_rotation=90)
    dot_chart.title = ''
    dot_chart.x_labels = labels
    dot_chart.add('DD', dd)
    dot_chart.add('NDTV', ndtv)
    dot_chart.add('Times', times)
    dot_chart.render_to_png(cht_nm)"""
    
    dot_chart = pygal.Dot(x_label_rotation=30,logarithmic=True,width=900,height=1000,style=BlueStyle)
    dot_chart.title = ''
    dot_chart.x_labels = labels
    for key in all_common.keys():
        dot_chart.add(key,all_common[key])
    dot_chart.render_to_png(cht_nm)

0
1


In [13]:
#data[0]
